In [ ]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
import zipfile
import os

# Step 1: Mount Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


# Data Preprocessing

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/실전프로젝트2/129.채용면접 인터뷰 데이터/01-1.정식개방데이터/Training/02.라벨링데이터/all_train_2.csv', encoding='utf-8')

In [ ]:
df_valid = pd.read_csv('/content/drive/MyDrive/실전프로젝트2/129.채용면접 인터뷰 데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/all_valid_2.csv', encoding='utf-8')

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
df_train.head()

,version,dataSet.info.date,dataSet.info.occupation,dataSet.info.channel,dataSet.info.place,dataSet.info.gender,dataSet.info.ageRange,dataSet.info.experience,dataSet.question.raw.text,dataSet.question.raw.wordCount,dataSet.question.emotion,dataSet.question.intent,dataSet.answer.raw.text,dataSet.answer.raw.wordCount,dataSet.answer.emotion,dataSet.answer.intent,dataSet.answer.summary.text,dataSet.answer.summary.wordCount
0,1.0,20230116,SM,MOCK,ONLINE,FEMALE,45-54,NEW,지원자님께서 이제 많은 친구분들이 있으실 텐데요 어 그중에 한 분만 골라서 이제 구...,20,[],[],저에게는 고등학교 때부터 알고 지낸 친구가 있습니다. 이 친구를 칭찬하고 싶네요. ...,92,[{'text': '그러다보니 주변에 사람이 많고 고민거리가 있어서 이야기를 나누면...,[{'text': '그래서 이 친구의 이런 편안함이 제가 오랫동안 벗으로 사귈 수 ...,체격이 튼튼하고 말씨도 부드럽고 서글서글한 인상의 고등학교 때부터 알고 지낸 친구를...,31
1,1.0,20230116,SM,MOCK,ONLINE,FEMALE,45-54,NEW,협업을 해야 할 일이 많이 생길 수도 있는데요 이전 직장에서나 대학에서 협업을 하셨...,18,[],[],협업을 할 때 어려운 점이 당연히 있을 수 밖에 없다고 생각합니다. 협엽이라고 하는...,160,[],"[{'text': '', 'expression': '', 'category': 'a...",협업은 혼자가 아닌 여러 명이 함께 할 수 있는 얘기를 하는 것이므로 시간적 조율이...,53
2,1.0,20230116,SM,MOCK,ONLINE,FEMALE,45-54,NEW,티비 라디오 인터넷 등 다양한 매체의 매체별 마케팅 전략의 차이에 대해 말씀해 주시...,13,[],[],매개체별 마케팅의 전략 차이가 그렇게 크게 나타난다고는 볼 수 없겠지만 티비는 말 ...,129,[],[{'text': '매개체별 마케팅의 전략 차이가 그렇게 크게 나타난다고는 볼 수 ...,티비는 리모콘 하나만 키면 화면으로 볼 수 있고 들을 수 있는 장점이 있어서 다양한...,47
3,1.0,20230116,SM,MOCK,ONLINE,FEMALE,55-,NEW,우리는 항상 보는 사람들이 있죠 왜 술을 마시면 꼭 진상들이 나타납니다 만약 고객사...,23,[],[],술을 마시다 보면 사람에 따라서 제각각의 형태를 봅니다. 가끔씩 고객분들이랑 술을 ...,55,[],"[{'text': '', 'expression': '', 'category': 'a...",고객분들과 술을 마시다 보면 진상적인 모습을 보이시기도 합니다. 하지만 고객도 제일...,22
4,1.0,20230116,SM,MOCK,ONLINE,FEMALE,35-44,NEW,기차가 운행 시에 산사태로 삼십 분 동안 정차되었습니다 만약 지원자님이 관리 책임자...,17,[],[],기차를 운행하는 중에 산사태로 삼십 분 정도 정차하는 상황이 생기고 제가 만약 관리...,56,[{'text': '기차를 운행하는 중에 산사태로 삼십 분 정도 정차하는 상황이 생...,"[{'text': '', 'expression': '', 'category': 'a...",관리 책임자라면 승객들의 안전 확보가 가장 중요하므로 대체 교통 수단을 확보하여 경...,26


In [ ]:
df_valid.head()

,version,dataSet.info.date,dataSet.info.occupation,dataSet.info.channel,dataSet.info.place,dataSet.info.gender,dataSet.info.ageRange,dataSet.info.experience,dataSet.question.raw.text,dataSet.question.raw.wordCount,dataSet.question.emotion,dataSet.question.intent,dataSet.answer.raw.text,dataSet.answer.raw.wordCount,dataSet.answer.emotion,dataSet.answer.intent,dataSet.answer.summary.text,dataSet.answer.summary.wordCount
0,1.0,20230116,ARD,MOCK,ONLINE,FEMALE,35-44,EXPERIENCED,네 협업하면서 지원자께서 가장 어려웠던 점은 무엇입니까,7,[],[],음 의견이 상충될 경우가 있는데 어 예를 들어서 마케팅 팀이 원하는 표지 디자인 혹...,94,[],[{'text': '음 의견이 상충될 경우가 있는데 어 예를 들어서 마케팅 팀이 원...,마케터는 다른 프로젝트나 다른 업무와 상충될 경우가 있기 때문에 업무에 대해서 얘기...,21
1,1.0,20230116,ARD,MOCK,ONLINE,FEMALE,55-,EXPERIENCED,구급 대원이 주취자에게 폭행을 당해 순직하셨는데 어떻게 생각하나,8,[],[],구급대원이 주취자에게 폭행을 당해 순직하셨는데 어떻게 생각하나요 라고 질문을 주셨는...,122,[],"[{'text': '', 'expression': '', 'category': 'a...",구급대원이 주취자에게 폭행을 당해 순직하셨는데 어떻게 생각하나요 라고 질문 주셨습니...,37
2,1.0,20230116,ARD,MOCK,ONLINE,FEMALE,55-,EXPERIENCED,어 내가 직무를 수행함에 있어서 어떤 뭐 모르는 과제도 나올 수 있겠죠 그런 문제에...,27,[],[],직무 수행 시 모르는 과제에 대해서 해결하는 방법은 무엇입니까 라고 질문을 주셨는데...,115,[],"[{'text': '', 'expression': '', 'category': 'a...",직무 수행 시 모르는 과제에 대한 해결방법에 대한 질문입니다. 모르는 일에 부딪힐 ...,39
3,1.0,20230116,ARD,MOCK,ONLINE,FEMALE,55-,EXPERIENCED,팀으로 일하는 것이 있구요 혼자 일하는 것이 있다면 그 둘 중에 내가 팀이 맞는지 ...,23,[],[],팀으로 일하는 것과 혼자 일하는 것 중 어느 것을 선호하는가 라고 질문을 해 주셨는...,132,"[{'text': '감사합니다.', 'expression': 'p-gratitude...",[{'text': '어 언제까지 혼자 일할 수 없기 때문에 저는 팀원으로서 또 이 ...,팀으로 일하는 것과 혼자 일하는 것 중 선호하는 것에 대한 질문을 해 주셨습니다. ...,43
4,1.0,20230116,ARD,MOCK,ONLINE,FEMALE,55-,EXPERIENCED,기차가 운행하고 있는 중간 산사태로 삼십 분 동안 정차가 되었습니다 이런 상황에서 ...,16,[],[],어 기차 운행 시 산사태로 삼십 분 동안 정차되었습니다. 이때 어떻게 조치 하실 건...,114,[],"[{'text': '', 'expression': '', 'category': 'a...",기차 운행 시 산사태로 정차되었을 때 어떻게 조치할 거냐는 질문에는 기관사 및 직원...,33


In [ ]:
df_train['dataSet.info.occupation'].unique()

array(['SM', 'PS', 'RND', 'ICT', 'ARD', 'MM', 'BM'], dtype=object)

In [ ]:
occupation = {'SM':'영업마케팅', 'PS':'공공서비스','RND':'연구개발', 'ICT':'정보통신','ARD':'디자인', 'MM':'생산관리', 'BM': '경영사무' }

df_train['dataSet.info.occupation'] = df_train['dataSet.info.occupation'].replace(occupation)

# 변경된 데이터 확인
print(df_train['dataSet.info.occupation'].head())

0    영업마케팅
1    영업마케팅
2    영업마케팅
3    영업마케팅
4    영업마케팅
Name: dataSet.info.occupation, dtype: object


In [ ]:
df_train.head()

,version,dataSet.info.date,dataSet.info.occupation,dataSet.info.channel,dataSet.info.place,dataSet.info.gender,dataSet.info.ageRange,dataSet.info.experience,dataSet.question.raw.text,dataSet.question.raw.wordCount,dataSet.question.emotion,dataSet.question.intent,dataSet.answer.raw.text,dataSet.answer.raw.wordCount,dataSet.answer.emotion,dataSet.answer.intent,dataSet.answer.summary.text,dataSet.answer.summary.wordCount
0,1.0,20230116,영업마케팅,MOCK,ONLINE,FEMALE,45-54,NEW,지원자님께서 이제 많은 친구분들이 있으실 텐데요 어 그중에 한 분만 골라서 이제 구...,20,[],[],저에게는 고등학교 때부터 알고 지낸 친구가 있습니다. 이 친구를 칭찬하고 싶네요. ...,92,[{'text': '그러다보니 주변에 사람이 많고 고민거리가 있어서 이야기를 나누면...,[{'text': '그래서 이 친구의 이런 편안함이 제가 오랫동안 벗으로 사귈 수 ...,체격이 튼튼하고 말씨도 부드럽고 서글서글한 인상의 고등학교 때부터 알고 지낸 친구를...,31
1,1.0,20230116,영업마케팅,MOCK,ONLINE,FEMALE,45-54,NEW,협업을 해야 할 일이 많이 생길 수도 있는데요 이전 직장에서나 대학에서 협업을 하셨...,18,[],[],협업을 할 때 어려운 점이 당연히 있을 수 밖에 없다고 생각합니다. 협엽이라고 하는...,160,[],"[{'text': '', 'expression': '', 'category': 'a...",협업은 혼자가 아닌 여러 명이 함께 할 수 있는 얘기를 하는 것이므로 시간적 조율이...,53
2,1.0,20230116,영업마케팅,MOCK,ONLINE,FEMALE,45-54,NEW,티비 라디오 인터넷 등 다양한 매체의 매체별 마케팅 전략의 차이에 대해 말씀해 주시...,13,[],[],매개체별 마케팅의 전략 차이가 그렇게 크게 나타난다고는 볼 수 없겠지만 티비는 말 ...,129,[],[{'text': '매개체별 마케팅의 전략 차이가 그렇게 크게 나타난다고는 볼 수 ...,티비는 리모콘 하나만 키면 화면으로 볼 수 있고 들을 수 있는 장점이 있어서 다양한...,47
3,1.0,20230116,영업마케팅,MOCK,ONLINE,FEMALE,55-,NEW,우리는 항상 보는 사람들이 있죠 왜 술을 마시면 꼭 진상들이 나타납니다 만약 고객사...,23,[],[],술을 마시다 보면 사람에 따라서 제각각의 형태를 봅니다. 가끔씩 고객분들이랑 술을 ...,55,[],"[{'text': '', 'expression': '', 'category': 'a...",고객분들과 술을 마시다 보면 진상적인 모습을 보이시기도 합니다. 하지만 고객도 제일...,22
4,1.0,20230116,영업마케팅,MOCK,ONLINE,FEMALE,35-44,NEW,기차가 운행 시에 산사태로 삼십 분 동안 정차되었습니다 만약 지원자님이 관리 책임자...,17,[],[],기차를 운행하는 중에 산사태로 삼십 분 정도 정차하는 상황이 생기고 제가 만약 관리...,56,[{'text': '기차를 운행하는 중에 산사태로 삼십 분 정도 정차하는 상황이 생...,"[{'text': '', 'expression': '', 'category': 'a...",관리 책임자라면 승객들의 안전 확보가 가장 중요하므로 대체 교통 수단을 확보하여 경...,26


In [ ]:
df_valid['dataSet.info.occupation'].unique()

array(['ARD', 'BM', 'MM', 'SM', 'RND', 'ICT', 'PS'], dtype=object)

In [ ]:
occupation = {'SM':'영업마케팅', 'PS':'공공서비스','RND':'연구개발', 'ICT':'정보통신','ARD':'디자인', 'MM':'생산관리', 'BM': '경영사무' }
# 컬럼의 값을 occupation 딕셔너리 내용으로 변경
df_valid['dataSet.info.occupation'] = df_valid['dataSet.info.occupation'].replace(occupation)

# 변경된 데이터 확인
print(df_valid['dataSet.info.occupation'].head())

0    디자인
1    디자인
2    디자인
3    디자인
4    디자인
Name: dataSet.info.occupation, dtype: object


In [ ]:
df_valid.head()

,version,dataSet.info.date,dataSet.info.occupation,dataSet.info.channel,dataSet.info.place,dataSet.info.gender,dataSet.info.ageRange,dataSet.info.experience,dataSet.question.raw.text,dataSet.question.raw.wordCount,dataSet.question.emotion,dataSet.question.intent,dataSet.answer.raw.text,dataSet.answer.raw.wordCount,dataSet.answer.emotion,dataSet.answer.intent,dataSet.answer.summary.text,dataSet.answer.summary.wordCount
0,1.0,20230116,디자인,MOCK,ONLINE,FEMALE,35-44,EXPERIENCED,네 협업하면서 지원자께서 가장 어려웠던 점은 무엇입니까,7,[],[],음 의견이 상충될 경우가 있는데 어 예를 들어서 마케팅 팀이 원하는 표지 디자인 혹...,94,[],[{'text': '음 의견이 상충될 경우가 있는데 어 예를 들어서 마케팅 팀이 원...,마케터는 다른 프로젝트나 다른 업무와 상충될 경우가 있기 때문에 업무에 대해서 얘기...,21
1,1.0,20230116,디자인,MOCK,ONLINE,FEMALE,55-,EXPERIENCED,구급 대원이 주취자에게 폭행을 당해 순직하셨는데 어떻게 생각하나,8,[],[],구급대원이 주취자에게 폭행을 당해 순직하셨는데 어떻게 생각하나요 라고 질문을 주셨는...,122,[],"[{'text': '', 'expression': '', 'category': 'a...",구급대원이 주취자에게 폭행을 당해 순직하셨는데 어떻게 생각하나요 라고 질문 주셨습니...,37
2,1.0,20230116,디자인,MOCK,ONLINE,FEMALE,55-,EXPERIENCED,어 내가 직무를 수행함에 있어서 어떤 뭐 모르는 과제도 나올 수 있겠죠 그런 문제에...,27,[],[],직무 수행 시 모르는 과제에 대해서 해결하는 방법은 무엇입니까 라고 질문을 주셨는데...,115,[],"[{'text': '', 'expression': '', 'category': 'a...",직무 수행 시 모르는 과제에 대한 해결방법에 대한 질문입니다. 모르는 일에 부딪힐 ...,39
3,1.0,20230116,디자인,MOCK,ONLINE,FEMALE,55-,EXPERIENCED,팀으로 일하는 것이 있구요 혼자 일하는 것이 있다면 그 둘 중에 내가 팀이 맞는지 ...,23,[],[],팀으로 일하는 것과 혼자 일하는 것 중 어느 것을 선호하는가 라고 질문을 해 주셨는...,132,"[{'text': '감사합니다.', 'expression': 'p-gratitude...",[{'text': '어 언제까지 혼자 일할 수 없기 때문에 저는 팀원으로서 또 이 ...,팀으로 일하는 것과 혼자 일하는 것 중 선호하는 것에 대한 질문을 해 주셨습니다. ...,43
4,1.0,20230116,디자인,MOCK,ONLINE,FEMALE,55-,EXPERIENCED,기차가 운행하고 있는 중간 산사태로 삼십 분 동안 정차가 되었습니다 이런 상황에서 ...,16,[],[],어 기차 운행 시 산사태로 삼십 분 동안 정차되었습니다. 이때 어떻게 조치 하실 건...,114,[],"[{'text': '', 'expression': '', 'category': 'a...",기차 운행 시 산사태로 정차되었을 때 어떻게 조치할 거냐는 질문에는 기관사 및 직원...,33


In [ ]:
import torch
from torch.utils.data import Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=256):
        self.qa_pairs = list(zip(dataframe['dataSet.question.raw.text'], dataframe['dataSet.answer.raw.text'], dataframe['dataSet.answer.summary.text']))
        self.veteran = dataframe['dataSet.info.experience'].apply(lambda x: '신입' if x == 'NEW' else '경력직')
        self.occupation = dataframe['dataSet.info.occupation']
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.qa_pairs)

    def __getitem__(self, idx):
        question, answer,summary = self.qa_pairs[idx]
        veteran = self.veteran[idx] # 어떻게 프롬프트에 추가할지
        occupation = self.occupation[idx] # 프롬프트 어떻게 바꿀지

        '''
        # 1. 기본 구조 유지 + veteran 추가
        input_text_1 = f"<s> 질문: {question} 경력: {veteran} 답변: {answer} </s>"

        # 2. 기본 구조 유지 + occupation 추가
        input_text_2 = f"<s> 질문: {question} 직업: {occupation} 답변: {answer} </s>"

        # 3. veteran + occupation 포함
        input_text_3 = f"<s> 질문: {question} 경력: {veteran} 직업: {occupation} 답변: {answer} </s>"

        # 4. 요약 정보 강조
        input_text_4 = f"<s> 질문: {question} 경력: {veteran} 직업: {occupation} 요약: {summary} 답변: {answer} </s>"

        # 5. 요약을 질문 직후로 배치
        input_text_5 = f"<s> 질문: {question} 요약: {summary} 경력: {veteran} 직업: {occupation} 답변: {answer} </s>"

        # 6. 역할 기반 강조
        input_text_6 = f"<s> 역할: {occupation} 경력 상태: {veteran} 질문: {question} 답변: {answer} </s>"

        # 7. 질문과 답변을 분리 강조
        input_text_7 = f"<s> 질문: {question} 경력: {veteran} 직업: {occupation} </s><s> 답변: {answer} </s>"

        # 8. veteran과 occupation 간 순서를 변경
        input_text_8 = f"<s> 질문: {question} 직업: {occupation} 경력: {veteran} 요약: {summary} 답변: {answer} </s>"
        '''
        input_text = f"<s> 질문: {question} 경력: {veteran} 직업: {occupation} 답변: {answer} </s>"
        encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        # labels를 input_ids의 복사본으로 생성
        labels = input_ids.clone()

        # 패딩 토큰에 대한 손실을 무시하기 위해 패딩 토큰 인덱스를 -100으로 설정
        labels[input_ids == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

train_dataset = QADataset(df_train, tokenizer)
valid_dataset = QADataset(df_valid, tokenizer)

In [ ]:
print(len(valid_dataset))

8028


In [ ]:
from torch.utils.data import DataLoader

# Dataset과 DataLoader 생성
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# DataLoader에서 배치 샘플 확인
for batch in train_dataloader:
    print(batch['input_ids'].shape)  # (batch_size, max_length)
    print(batch['attention_mask'].shape)  # (batch_size, max_length)
    print(batch['labels'].shape)  # (batch_size, max_length)
    for i in batch['input_ids']:
      print(tokenizer.decode(i, skip_special_tokens=True))
      break
    break

torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])
질문: 협업하는 과제 혼자 하는 과제 둘 중에서 어떤 과제를 더 선호하시나요 경력: 신입 직업: 생산관리 답변: 네 당연히 저는 협업하는 과제를 더 선호합니다. 그 이유는 혼자 하는 과제는 인제 혼자 그 알아서 시간 관리와 어떤 업무 추진을 하는 데는 매우 좀 부드럽고 어 효율적으로 할 수 있겠죠. 하지만은 혼자 했을 때는 아무래도 인제 그 좀 독선적이고 주관적이고 어떤 그런 경우가 발생할 수 있기 때문에 혹시나 어떤 나온 그 과제 결과가 향후에 어떤 그 현실화 한다든지 상품화 한다든지 그 과정에 있어서 쪼금 시행착오를 겪는 그런 과정이 좀 많이 나올 수 있는 리스크가 좀 높다 저는 그런 생각을 좀 하고요. 협업을 했을 때는 그 여러 분야의 전문가 아 전문가가 경험 있는 분들하고 같이 공동으로 추진하기 때문에 혼자 했을 때는 한 가지 방법을 생각하겠지만은 협업을 했을 때는 여러 가지 방법을 두고 같이 그 중에서 가장 좋은 우수한 답을 찾고 그 과정 가장 빠른 지름길을 찾고 하는 그런 것을 인제 같이 그 진행을 하기 때문에 일단 서로의 어떤 경험과 지식을 공유하는 그런 중요한 그 학습의 과정이 될 수가 있고 두 번째로는 그 협업에서 나오는 과제가 실질적으로 나중에 시장에 반영이 됐을 때 업무에 반영이 됐을 때 시행착오를 줄일 수 있는 또 그런 계기가 되기 때문에 저는 당연히 협업이 더


In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.3 MB/s eta 0:00:00


## Model Train

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from tqdm import tqdm
from bert_score import score  # bert-score import 추가

# SmoothingFunction 설정
smoothing = SmoothingFunction().method1

# 모델 학습 설정
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 옵티마이저 및 스케줄러 정의
optimizer = AdamW(model.parameters(), lr=5e-5)  # 옵티마이저 정의
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)  # 스케줄러 정의

# ROUGE 계산기
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
best_valid_loss = float('inf')

# 학습 및 검증 루프
for epoch in range(num_epochs):
    # 학습 단계
    model.train()
    total_train_loss = 0
    train_steps = 0
    total_train_bleu = 0
    total_train_rouge = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}

    train_pbar = tqdm(train_dataloader, desc=f'Training Epoch {epoch+1}')
    for batch in train_pbar:
        # 입력 데이터 준비
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 모델 출력
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # 역전파 및 최적화
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()  # 옵티마이저 업데이트
        scheduler.step()  # 스케줄러 업데이트
        optimizer.zero_grad()

        # 손실 누적
        total_train_loss += loss.item()
        train_steps += 1

        # BLEU 및 ROUGE 계산
        generated_ids = outputs.logits.argmax(dim=-1)
        generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        labels_for_decoding = labels.masked_fill(labels == -100, tokenizer.pad_token_id)
        reference_texts = tokenizer.batch_decode(labels_for_decoding, skip_special_tokens=True)

        for ref, gen in zip(reference_texts, generated_texts):
            bleu_score = sentence_bleu([ref.split()], gen.split(), smoothing_function=smoothing)
            total_train_bleu += bleu_score

            rouge_scores = scorer.score(ref, gen)
            for key in total_train_rouge.keys():
                total_train_rouge[key] += rouge_scores[key].fmeasure

        train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    # 평균 점수 계산
    avg_train_loss = total_train_loss / train_steps
    avg_train_bleu = total_train_bleu / train_steps
    avg_train_rouge = {k: v / train_steps for k, v in total_train_rouge.items()}

    print(f"\nEpoch {epoch+1} Average Training Loss: {avg_train_loss:.4f}")
    print(f"Epoch {epoch+1} Average Training BLEU: {avg_train_bleu:.4f}")
    print(f"Epoch {epoch+1} Average Training ROUGE: {avg_train_rouge}")

    # 검증 단계
    model.eval()
    total_valid_loss = 0
    valid_steps = 0
    total_valid_bleu = 0
    total_valid_rouge = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}

    all_generated_texts = []  # BERTScore를 계산하기 위한 생성 텍스트 리스트
    all_reference_texts = []  # BERTScore를 계산하기 위한 참조 텍스트 리스트

    with torch.no_grad():
        valid_pbar = tqdm(valid_dataloader, desc=f'Validation Epoch {epoch+1}')
        for batch in valid_pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # 손실 누적
            total_valid_loss += loss.item()
            valid_steps += 1

            # BLEU 및 ROUGE 계산
            generated_ids = outputs.logits.argmax(dim=-1)
            generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            labels_for_decoding = labels.masked_fill(labels == -100, tokenizer.pad_token_id)
            reference_texts = tokenizer.batch_decode(labels_for_decoding, skip_special_tokens=True)

            all_generated_texts.extend(generated_texts)
            all_reference_texts.extend(reference_texts)

            for ref, gen in zip(reference_texts, generated_texts):
                bleu_score = sentence_bleu([ref.split()], gen.split(), smoothing_function=smoothing)
                total_valid_bleu += bleu_score

                rouge_scores = scorer.score(ref, gen)
                for key in total_valid_rouge.keys():
                    total_valid_rouge[key] += rouge_scores[key].fmeasure

            valid_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    # 평균 점수 계산
    avg_valid_loss = total_valid_loss / valid_steps
    avg_valid_bleu = total_valid_bleu / valid_steps
    avg_valid_rouge = {k: v / valid_steps for k, v in total_valid_rouge.items()}

    # BERTScore 계산
    P, R, F1 = score(all_generated_texts, all_reference_texts, lang="ko", verbose=True)  # lang="ko"는 한국어 기준

    print(f"Epoch {epoch+1} Average Validation Loss: {avg_valid_loss:.4f}")
    print(f"Epoch {epoch+1} Average Validation BLEU: {avg_valid_bleu:.4f}")
    print(f"Epoch {epoch+1} Average Validation ROUGE: {avg_valid_rouge}")
    print(f"Epoch {epoch+1} BERTScore Precision: {P.mean():.4f}")
    print(f"Epoch {epoch+1} BERTScore Recall: {R.mean():.4f}")
    print(f"Epoch {epoch+1} BERTScore F1: {F1.mean():.4f}")

    # 최적의 모델 저장
    if avg_valid_loss < best_valid_loss:
        best_valid_loss = avg_valid_loss
        print(f"Saving best model with validation loss: {best_valid_loss:.4f}")
        torch.save(model.state_dict(), '/content/drive/MyDrive/실전프로젝트2/best_model_3.pt')

print("\nTraining completed!")


Training Epoch 1: 100%|██████████| 2086/2086 [17:46<00:00,  1.96it/s, loss=2.9899]



Epoch 1 Average Training Loss: 3.0451
Epoch 1 Average Training BLEU: 2.2733
Epoch 1 Average Training ROUGE: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}


Validation Epoch 1: 100%|██████████| 251/251 [00:52<00:00,  4.79it/s, loss=2.9493]


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/251 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/126 [00:00<?, ?it/s]

done in 110.64 seconds, 72.56 sentences/sec
Epoch 5 Average Validation Loss: 2.9987
Epoch 5 Average Validation BLEU: 2.1367
Epoch 5 Average Validation ROUGE: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}
Epoch 5 BERTScore Precision: 0.7580
Epoch 5 BERTScore Recall: 0.7689
Epoch 5 BERTScore F1: 0.7629
Saving best model with validation loss: 2.9987

Training completed!


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load the tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                              bos_token='</s>',
                                              eos_token='</s>',
                                              unk_token='<unk>',
                                              pad_token='<pad>',
                                              mask_token='<mask>')

# Load the pretrained model
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
model.eval()

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate a response
def generate_response(prompt, max_length=200):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Example prompt
prompt = "<s> 질문: 자신의 강점은 무엇이라고 생각합니까? 답변:"
print("Prompt:", prompt)

# 1. Generate response with pretrained weights only
print("\n--- Response with Pretrained Weights ---")
response_pretrained = generate_response(prompt)
print(response_pretrained.split('답변')[1])

# 2. Load custom weights and generate response again
model.load_state_dict(torch.load('/content/drive/MyDrive/실전프로젝트2/best_model_3.pt', map_location=torch.device('cpu')))
model.eval()

print("\n--- Response with Custom Weights (best_model.pt) ---")
response_custom = generate_response(prompt)
print(response_custom.split('답변')[1])


Prompt: <s> 질문: 자신의 강점은 무엇이라고 생각합니까? 답변:

--- Response with Pretrained Weights ---
: 나는 강점이란 것은 '강점'이라는 것입니다."
두 사람이 같이 답한 것은 우연한 만남, 즉 만남에 있어서 공통점이 있기 때문이었다.
이것은 곧 우연이었다.
그런데 두 사람이 함께 대답한 것이 문제였다.
그것은 우연이 아니라 우연에 의한 것이었다.
두 사람은 서로의 강점을 확인하기 위하여 서로의 얼굴을 마주보고 마주한 것이었다.
둘이 같이 마주앉은 것은 그 만남에서 우연의 영향이 컸기 때문이었다.
또 한 가지는 우연과 우연을 결합시키는 것이 무척 중요하다는 것을 알고 있었기 때문이다.
사실 우연은 그 자체가 하나의 실체에 지나지 않는다.
이것을 우연으로 표현하면 '우연'이란 것을 의미한다.
우연의 실체는 우연에서 유래한다는 것, 즉 우연성이 존재하는 것이다.
우연은 우연도 아니고 우연성이다.
우연이란 것은 우리가 상상할 수 있는 것을 말한다.
우연을 통해서 우리는 이 실체가 무엇인가를 확인할 수 있다.
우연이 존재하는 이유는 우리의 마음을 움직이는 힘이 있기 때문이다.
우연과 사랑을 통해 우리는 우리의 마음이 움직이게 되고


<ipython-input-44-5ea51c38d619>:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/실전프로젝트2/best_model_3.pt',


--- Response with Custom Weights (best_model.pt) ---
: 저의 강점과 단점을 말씀해 주시고 그 장점을 특화시키고 단점 극복하기 위한 노력도 함께 말씀 부탁드립니다 경력: 신입 직업: 공공서비스 답: 어 저는 어떤 일이든지 빠르게 적응하는 편입니다. 그게 저한테 가장 큰 장점이라고 할 수 있겠는데요. 저에 성격은 어 굉장히 외향적이고 긍정적인 성격을 갖고 있습니다. 어 그런 성격은 어떤 상황에도 흔들리지 않고 또 그 일이 어떤 일인지에 대해서 빠르게 판단하는 편입니다, 그래서 어떤 일을 할 때 어 그것이 저한테는 굉장한 장점이 되곤 합니다. 어 하지만 어 단점으로 생각을 한다면 제가 조금은 어 조금 소심하고 또 일을 빨리 하는 편인데요. 어 그러다 보니까 다른 사람들에게 이야기를 잘 하지 않습니다. 어 어떤 사안이 발생이 됐을 때 그것을 빨리 결정하지 못하고 어 다른 사람들이 어 결정을 할 때도 어 제가 결정을 빨리 하지 못하고 또 저 혼자 고민하는 경우가 많습니다. 그래서 제가 어떤 일에


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load the tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                              bos_token='</s>',
                                              eos_token='</s>',
                                              unk_token='<unk>',
                                              pad_token='<pad>',
                                              mask_token='<mask>')

# Load the pretrained model
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
model.eval()

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate a response
def generate_response(prompt, max_length=200):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Example prompt
prompt = "<s> 질문: 자신의 강점은 무엇이라고 생각합니까? 경력: 경력 직업: 경영관리 답변:"
print("Prompt:", prompt)

# 1. Generate response with pretrained weights only
print("\n--- Response with Pretrained Weights ---")
response_pretrained = generate_response(prompt)
print(response_pretrained.split('답변')[1])

# 2. Load custom weights and generate response again
model.load_state_dict(torch.load('/content/drive/MyDrive/실전프로젝트2/best_model_3.pt', map_location=torch.device('cpu')))
model.eval()

print("\n--- Response with Custom Weights (best_model.pt) ---")
response_custom = generate_response(prompt)
print(response_custom.split('답변')[1])


Prompt: <s> 질문: 자신의 강점은 무엇이라고 생각합니까? 경력: 경력 직업: 경영관리 답변:

--- Response with Pretrained Weights ---
: 자신이 속한 회사의 경영자를 좋아한다고 생각합니다."
"그렇다면 그 회사에서 얼마나 일하고 있는지, 그리고 그 회사의 직원들은 어떻게 일하는지를?"
그리고 그들의 이야기를 듣고 나서 그는 그 회사에 대해 이렇게 말했다
"그래서 당신은 이 회사의 직원과 회사를 더 잘 알고 있어야 합니다."
이런 질문도 들어본 적이 없었다.
그가 이 회사에서 일한 이유는 무엇일까?
"아마도 나는 이 회사에 더 많이 일하기 위해서일 겁니다."
나는 이렇게 대답했다.
"나는 당신이 하는 일에 대해서는 아무도 알지 못하는 것 같군. 나는 당신만의 방식으로 당신을 도와주겠어."
"네, 그렇습니다."
그러나 실제로 그 회사는 당신에게 정말 잘 알려진 기업이다.
그 회사는 또 하나의 훌륭한 기업이기도 하다.
그것은 바로 '스스로를 신뢰하는 경영'이다.
그 기업들은 스스로에게 신뢰한다는 것이다.
이렇게 신뢰하기 위해서는 어떻게 해야 할까?
그들이 당신과 같은 환경에서 일하는 이유는 무엇인가?
그들은


<ipython-input-45-c807a8fde29c>:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/실전프로젝트2/best_model_3.pt',


--- Response with Custom Weights (best_model.pt) ---
: 제가 생각하는 강점과 단점은 각각 다릅니다. 일단 저의 강점이자 단점이기도 하지만 단점이 될 수도 있다고 생각합니다. 저는 어릴 때부터 다른 사람을 굉장히 많이 도와준 적이 있습니다. 특히 어린 시절에 저희 엄마는 저에게 굉장한 학비를 대주고 저희는 그 학기 동안에만 그 엄마에게 많은 도움을 줬기 때문에 저에게는 그만한 친구가 없었습니다. 제가 다른 사람에게 굉장이 많이 의지하고 의지하기 때문에 다른 사람들이 저를 신뢰하지 않을 수 있습니다. 하지만 저 같은 경우 다른 사람의 도움을 굉장의 노력을 통해서 제가 모르는 것들을 제가 알고 있는 것들을 저 스스로 깨닫고 그것들을 보완해 나가는 능력이 있습니다. 그래서 다른 사람들에 비해서 제가 가진 강점에 대해서 저도 많이 이야기를 하고 어 제가 가지고 있는 강점으로 사람들에게 도움을 많이 줘서 제가 부족한 부분이나 저가 생각하는 것들을 보완하고 또 그 사람들의 어떤 문제점들을 해결해 주고 다른 사람들에게 도움이 된다면 어 저 자신도 굉


## Validation

In [ ]:
# 모델 초기화
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")  # 모델 구조 로드
model.resize_token_embeddings(len(tokenizer))  # 토크나이저 크기 맞춤
model.load_state_dict(torch.load('/content/drive/MyDrive/실전프로젝트2/best_model_4.pt'))  # 가중치 로드
model = model.to(device)  # 모델을 GPU 또는 CPU로 이동
model.eval()  # 평가 모드 전환


<ipython-input-19-883425b9bbcd>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/실전프로젝트2/best_model_4.pt'))

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
!pip install korouge_score

In [ ]:
from korouge_score import rouge_scorer
from bert_score import score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm

# SmoothingFunction 설정
smoothing = SmoothingFunction().method1

# ROUGE 및 BLEU 계산을 위한 준비
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL", "rougeLsum"])

# Validation 실행
total_valid_loss = 0
valid_steps = 0
total_valid_bleu = 0
total_valid_rouge = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
all_generated_texts = []
all_reference_texts = []

with torch.no_grad():
    valid_pbar = tqdm(valid_dataloader, desc='Validation')
    for batch in valid_pbar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 모델 출력
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # 손실 누적
        total_valid_loss += loss.item()
        valid_steps += 1

        # BLEU 및 ROUGE 계산
        generated_ids = outputs.logits.argmax(dim=-1)
        generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        labels_for_decoding = labels.masked_fill(labels == -100, tokenizer.pad_token_id)
        reference_texts = tokenizer.batch_decode(labels_for_decoding, skip_special_tokens=True)

        all_generated_texts.extend(generated_texts)
        all_reference_texts.extend(reference_texts)

        for ref, gen in zip(reference_texts, generated_texts):
            # 빈 텍스트 확인 및 제외
            if not ref.strip() or not gen.strip():
                continue

            # BLEU 점수 계산
            bleu_score = sentence_bleu([ref.split()], gen.split(), smoothing_function=smoothing)
            total_valid_bleu += bleu_score

            # ROUGE 점수 계산 및 누적
            rouge_scores = scorer.score(ref, gen)
            for key in total_valid_rouge.keys():
                total_valid_rouge[key] += rouge_scores[key].fmeasure

        valid_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

# 평균 점수 계산
avg_valid_loss = total_valid_loss / valid_steps
avg_valid_bleu = total_valid_bleu / valid_steps
avg_valid_rouge = {k: v / 8028 for k, v in total_valid_rouge.items()}

# BERTScore 계산
P, R, F1 = score(all_generated_texts, all_reference_texts, lang="ko", verbose=True)

# 결과 출력
print(f"Average Validation Loss: {avg_valid_loss:.4f}")
print(f"Average Validation BLEU: {avg_valid_bleu:.4f}")
print(f"Average Validation ROUGE: {avg_valid_rouge}")
print(f"BERTScore Precision: {P.mean():.4f}")
print(f"BERTScore Recall: {R.mean():.4f}")
print(f"BERTScore F1: {F1.mean():.4f}")


Validation: 100%|██████████| 251/251 [03:05<00:00,  1.35it/s, loss=2.8606]


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/251 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/126 [00:00<?, ?it/s]

done in 115.30 seconds, 69.63 sentences/sec
Average Validation Loss: 3.0023
Average Validation BLEU: 1.7672
Average Validation ROUGE: {'rouge1': 0.3065894424121382, 'rouge2': 0.09108117509414317, 'rougeL': 0.2421079120163181}
BERTScore Precision: 0.7213
BERTScore Recall: 0.7598
BERTScore F1: 0.7390


In [ ]:
rouge_list

[{'rouge1': Score(precision=0.28431372549019607, recall=0.27102803738317754, fmeasure=0.277511961722488),
  'rouge2': Score(precision=0.0594059405940594, recall=0.05660377358490566, fmeasure=0.057971014492753624),
  'rougeL': Score(precision=0.23529411764705882, recall=0.22429906542056074, fmeasure=0.22966507177033493),
  'rougeLsum': Score(precision=0.23529411764705882, recall=0.22429906542056074, fmeasure=0.22966507177033493)},
 {'rouge1': Score(precision=0.3939393939393939, recall=0.3795620437956204, fmeasure=0.38661710037174724),
  'rouge2': Score(precision=0.0916030534351145, recall=0.08823529411764706, fmeasure=0.0898876404494382),
  'rougeL': Score(precision=0.2878787878787879, recall=0.2773722627737226, fmeasure=0.28252788104089216),
  'rougeLsum': Score(precision=0.2878787878787879, recall=0.2773722627737226, fmeasure=0.28252788104089216)},
 {'rouge1': Score(precision=0.2953020134228188, recall=0.2953020134228188, fmeasure=0.2953020134228188),
  'rouge2': Score(precision=0.040

In [ ]:
x,y,z = [],[],[]
for a in rouge_list:
  x.append(a['rouge1'].fmeasure)
  y.append(a['rouge2'].fmeasure)
  z.append(a['rougeL'].fmeasure)
print(len(x),len(y),len(z))
mx = sum(x)/len(x)
my = sum(y)/len(y)
mz = sum(z)/len(z)
print(mx,my,mz)

8028 8028 8028
0.3065894424121382 0.09108117509414317 0.2421079120163181


In [ ]:
rouge_scores

{'rouge1': Score(precision=0.3163841807909605, recall=0.4057971014492754, fmeasure=0.35555555555555557),
 'rouge2': Score(precision=0.09659090909090909, recall=0.12408759124087591, fmeasure=0.1086261980830671),
 'rougeL': Score(precision=0.2711864406779661, recall=0.34782608695652173, fmeasure=0.3047619047619048),
 'rougeLsum': Score(precision=0.2711864406779661, recall=0.34782608695652173, fmeasure=0.3047619047619048)}